In [1]:
# CELL 1: Setup
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🐺 FENRIR MISSION: THE DAY BEFORE")
print("="*60)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("\nObjective: Can we buy Day 0 and sell into the Day 1 gap?")
print("\nThis is INVESTIGATION, not celebration.")

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🐺 FENRIR MISSION: THE DAY BEFORE
Analysis Date: 2026-01-06 23:39

Objective: Can we buy Day 0 and sell into the Day 1 gap?

This is INVESTIGATION, not celebration.


In [2]:
# CELL 2: Universe Definition
UNIVERSE = {
    'Quantum': ['IONQ', 'RGTI', 'QBTS', 'QUBT'],
    'Nuclear': ['CCJ', 'UUUU', 'LEU', 'OKLO', 'SMR'],
    'Space': ['RKLB', 'LUNR', 'ASTS', 'RDW', 'SIDU'],
    'Semi': ['NVDA', 'MU', 'AEVA', 'LAZR', 'NVTS', 'SKYT'],
    'AI_Infra': ['VRT', 'SMCI', 'LITE'],
    'Rare_Earth': ['MP', 'USAR']
}

ALL_TICKERS = [t for tickers in UNIVERSE.values() for t in tickers]
print(f"Universe: {len(ALL_TICKERS)} tickers")

Universe: 25 tickers


In [3]:
# CELL 3: Fetch Daily Data
def fetch_daily_data(ticker, period='6mo'):
    """Fetch daily OHLCV data"""
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(period=period, interval='1d')
        if df.empty:
            return None
        
        df['Return'] = df['Close'].pct_change() * 100
        df['Prev_Close'] = df['Close'].shift(1)
        df['Gap_Pct'] = ((df['Open'] - df['Prev_Close']) / df['Prev_Close']) * 100
        df['Intraday_Return'] = ((df['Close'] - df['Open']) / df['Open']) * 100
        df['Avg_Volume'] = df['Volume'].rolling(20).mean()
        df['Vol_Ratio'] = df['Volume'] / df['Avg_Volume']
        df['Close_Position'] = (df['Close'] - df['Low']) / (df['High'] - df['Low'])
        
        return df
    except Exception as e:
        return None

# Fetch all
print("Fetching data...")
all_data = {}
for ticker in ALL_TICKERS:
    df = fetch_daily_data(ticker)
    if df is not None and len(df) > 30:
        all_data[ticker] = df
        
print(f"Loaded {len(all_data)} tickers")

Fetching data...
Loaded 25 tickers


In [4]:
# CELL 4: Find All Runs and Their Day 0's
def find_runs_with_context(df, min_gain=10, min_days=2):
    """Find runs and capture Day -2, Day -1, Day 0, Day 1, etc."""
    runs = []
    
    i = 0
    while i < len(df) - 1:
        # Look for start of run (green day after red/flat)
        if df['Return'].iloc[i] > 2:  # Day 1 starts with 2%+ gain
            start_idx = i
            start_price = df['Open'].iloc[i]
            
            # Extend run
            j = i + 1
            red_count = 0
            while j < len(df) and red_count < 2:
                if df['Return'].iloc[j] < -1:
                    red_count += 1
                else:
                    red_count = 0
                j += 1
            
            end_idx = j - 2 if red_count >= 2 else j - 1
            end_idx = min(end_idx, len(df) - 1)
            
            if end_idx > start_idx:
                end_price = df['Close'].iloc[end_idx]
                run_gain = ((end_price - start_price) / start_price) * 100
                run_days = end_idx - start_idx + 1
                
                if run_gain >= min_gain and run_days >= min_days:
                    # Capture context - Day 0 is the day BEFORE Day 1
                    run_data = {
                        'day1_idx': start_idx,
                        'end_idx': end_idx,
                        'run_gain': run_gain,
                        'run_days': run_days,
                        'day1_date': df.index[start_idx],
                        
                        # Day 1 data
                        'day1_gap': df['Gap_Pct'].iloc[start_idx],
                        'day1_open': df['Open'].iloc[start_idx],
                        'day1_close': df['Close'].iloc[start_idx],
                        'day1_return': df['Return'].iloc[start_idx],
                        'day1_intraday': df['Intraday_Return'].iloc[start_idx],
                    }
                    
                    # Day 0 (day before Day 1)
                    if start_idx >= 1:
                        d0 = start_idx - 1
                        run_data['day0_date'] = df.index[d0]
                        run_data['day0_open'] = df['Open'].iloc[d0]
                        run_data['day0_close'] = df['Close'].iloc[d0]
                        run_data['day0_return'] = df['Return'].iloc[d0]
                        run_data['day0_vol_ratio'] = df['Vol_Ratio'].iloc[d0]
                        run_data['day0_close_position'] = df['Close_Position'].iloc[d0]
                        run_data['day0_high'] = df['High'].iloc[d0]
                        run_data['day0_low'] = df['Low'].iloc[d0]
                    
                    # Day -1 (2 days before Day 1)
                    if start_idx >= 2:
                        dm1 = start_idx - 2
                        run_data['daym1_return'] = df['Return'].iloc[dm1]
                        run_data['daym1_close'] = df['Close'].iloc[dm1]
                    
                    runs.append(run_data)
            
            i = end_idx + 1
        else:
            i += 1
    
    return runs

# Find all runs with context
all_runs = []
for ticker, df in all_data.items():
    runs = find_runs_with_context(df)
    for run in runs:
        run['ticker'] = ticker
    all_runs.extend(runs)

runs_df = pd.DataFrame(all_runs)
print(f"\n🎯 Found {len(runs_df)} total runs to analyze")
print(f"\nSample of runs:")
print(runs_df[['ticker', 'day1_date', 'run_gain', 'run_days', 'day1_gap']].head(10))


🎯 Found 125 total runs to analyze

Sample of runs:
  ticker                 day1_date    run_gain  run_days  day1_gap
0   IONQ 2025-08-22 00:00:00-04:00   99.083568        23 -0.188323
1   IONQ 2025-10-01 00:00:00-04:00   26.333803        10 -0.186995
2   IONQ 2025-11-14 00:00:00-05:00   11.400655         4 -5.330401
3   IONQ 2025-11-24 00:00:00-05:00   17.777774        14  2.493409
4   IONQ 2026-01-02 00:00:00-05:00   10.323843         3  2.540672
5   RGTI 2025-07-14 00:00:00-04:00   30.731705         6  0.985221
6   RGTI 2025-08-22 00:00:00-04:00   14.416631         6 -0.595655
7   RGTI 2025-09-09 00:00:00-04:00  104.459018        14  0.660069
8   RGTI 2025-10-02 00:00:00-04:00   56.406916        11  2.747068
9   RGTI 2026-01-02 00:00:00-05:00   12.499999         3  1.851015


---
# TEST 1: ENTRY TIMING - IS THE GAP A TRAP?

**Question:** If you bought at previous close (Day 0) vs open (Day 1), which is better?

In [5]:
# CELL 5: TEST 1 - Entry Timing Comparison

print("\n" + "="*70)
print("🔬 TEST 1: ENTRY TIMING - IS THE GAP A TRAP?")
print("="*70)

# Filter runs with Day 0 data
valid_runs = runs_df.dropna(subset=['day0_close', 'day1_open', 'day1_close'])
print(f"\nAnalyzing {len(valid_runs)} runs with complete Day 0/Day 1 data")

# Calculate returns from different entry points
results = []

for _, run in valid_runs.iterrows():
    # Entry points
    day0_close = run['day0_close']  # Buy at 4 PM day before
    day1_open = run['day1_open']    # Buy at 9:30 AM
    day1_close = run['day1_close']  # End of Day 1
    
    # Calculate returns to Day 1 close from each entry
    ret_from_day0_close = ((day1_close - day0_close) / day0_close) * 100
    ret_from_day1_open = ((day1_close - day1_open) / day1_open) * 100
    
    # The gap itself (Day 0 close → Day 1 open)
    gap_capture = ((day1_open - day0_close) / day0_close) * 100
    
    results.append({
        'ticker': run['ticker'],
        'run_gain': run['run_gain'],
        'day1_gap': run['day1_gap'],
        'day1_intraday': run['day1_intraday'],
        'ret_from_day0_close': ret_from_day0_close,  # Buy Day 0 4PM, sell Day 1 4PM
        'ret_from_day1_open': ret_from_day1_open,    # Buy Day 1 9:30, sell Day 1 4PM
        'gap_capture': gap_capture,                   # The gap itself
    })

timing_df = pd.DataFrame(results)

print(f"\n" + "="*70)
print("ENTRY TIMING ANALYSIS")
print("="*70)
print(f"\nSample Size: {len(timing_df)} runs\n")

print("| Entry Point | Avg Return to Day 1 Close | Win Rate | Median |")
print("|-------------|---------------------------|----------|--------|")

# Day 0 Close entry
d0_ret = timing_df['ret_from_day0_close']
d0_win = (d0_ret > 0).mean() * 100
print(f"| Day 0 Close (4 PM)  | {d0_ret.mean():+.1f}%                      | {d0_win:.0f}%      | {d0_ret.median():+.1f}%  |")

# Day 1 Open entry
d1_ret = timing_df['ret_from_day1_open']
d1_win = (d1_ret > 0).mean() * 100
print(f"| Day 1 Open (9:30 AM)| {d1_ret.mean():+.1f}%                      | {d1_win:.0f}%      | {d1_ret.median():+.1f}%  |")

# The gap
gap = timing_df['gap_capture']
print(f"| The Gap Itself      | {gap.mean():+.1f}%                      | {(gap > 0).mean()*100:.0f}%      | {gap.median():+.1f}%  |")

print(f"\n" + "-"*70)
print("KEY INSIGHT: THE GAP")
print("-"*70)
print(f"\nAvg Gap (Day 0 close → Day 1 open): {gap.mean():+.1f}%")
print(f"Gaps that were positive: {(gap > 0).mean()*100:.0f}%")
print(f"\nIf you bought Day 0 close and sold Day 1 OPEN:")
print(f"  You would capture the gap: {gap.mean():+.1f}% average")
print(f"  BEFORE the intraday fade begins.")

print(f"\n" + "-"*70)
print("THE INTRADAY FADE")
print("-"*70)
intraday = timing_df['day1_intraday']
print(f"\nAvg Day 1 Intraday Return (Open → Close): {intraday.mean():+.1f}%")
print(f"Day 1's that faded (intraday negative): {(intraday < 0).mean()*100:.0f}%")

# The verdict
print(f"\n" + "="*70)
print("🎯 VERDICT: IS THE GAP A TRAP?")
print("="*70)

advantage = d0_ret.mean() - d1_ret.mean()
print(f"\nDay 0 Close entry beats Day 1 Open entry by: {advantage:+.1f}%")

if advantage > 3:
    print(f"\n✅ YES - Buying Day 0 close is SIGNIFICANTLY better!")
    print(f"   The gap IS a trap for Day 1 buyers.")
elif advantage > 0:
    print(f"\n⚠️ SLIGHT EDGE - Day 0 entry is marginally better.")
else:
    print(f"\n❌ NO - Entry timing doesn't matter much.")


🔬 TEST 1: ENTRY TIMING - IS THE GAP A TRAP?

Analyzing 125 runs with complete Day 0/Day 1 data

ENTRY TIMING ANALYSIS

Sample Size: 125 runs

| Entry Point | Avg Return to Day 1 Close | Win Rate | Median |
|-------------|---------------------------|----------|--------|
| Day 0 Close (4 PM)  | +6.8%                      | 100%      | +4.8%  |
| Day 1 Open (9:30 AM)| +5.1%                      | 94%      | +4.3%  |
| The Gap Itself      | +1.7%                      | 71%      | +1.0%  |

----------------------------------------------------------------------
KEY INSIGHT: THE GAP
----------------------------------------------------------------------

Avg Gap (Day 0 close → Day 1 open): +1.7%
Gaps that were positive: 71%

If you bought Day 0 close and sold Day 1 OPEN:
  You would capture the gap: +1.7% average
  BEFORE the intraday fade begins.

----------------------------------------------------------------------
THE INTRADAY FADE
---------------------------------------------------------

---
# TEST 2: WHAT DOES DAY 0 LOOK LIKE?

**Question:** What patterns appear on the day BEFORE a run?

In [6]:
# CELL 6: TEST 2 - Day 0 Characteristics

print("\n" + "="*70)
print("🔬 TEST 2: WHAT DOES DAY 0 LOOK LIKE?")
print("="*70)

valid = runs_df.dropna(subset=['day0_return', 'day0_vol_ratio', 'day0_close_position'])
print(f"\nAnalyzing {len(valid)} Day 0's before confirmed runs")

print(f"\n" + "-"*70)
print("DAY 0 PRICE ACTION")
print("-"*70)

# Is Day 0 red?
day0_red = (valid['day0_return'] < 0).mean() * 100
print(f"\nDay 0 is RED: {day0_red:.0f}%")
print(f"Avg Day 0 return: {valid['day0_return'].mean():.1f}%")

# Where does Day 0 close in its range?
close_pos = valid['day0_close_position'].mean()
close_low = (valid['day0_close_position'] < 0.3).mean() * 100  # Closed in bottom 30%
close_high = (valid['day0_close_position'] > 0.7).mean() * 100  # Closed in top 30%
print(f"\nDay 0 Close Position:")
print(f"  Avg position in range: {close_pos:.1%} (0=low, 1=high)")
print(f"  Closed in bottom 30%: {close_low:.0f}%")
print(f"  Closed in TOP 30%: {close_high:.0f}%")

print(f"\n" + "-"*70)
print("DAY 0 VOLUME")
print("-"*70)

vol_elevated = (valid['day0_vol_ratio'] > 1.5).mean() * 100
vol_spike = (valid['day0_vol_ratio'] > 2.0).mean() * 100
print(f"\nDay 0 volume > 1.5x average: {vol_elevated:.0f}%")
print(f"Day 0 volume > 2.0x average: {vol_spike:.0f}%")
print(f"Avg Day 0 volume ratio: {valid['day0_vol_ratio'].mean():.1f}x")

print(f"\n" + "-"*70)
print("COMBINED SIGNALS ON DAY 0")
print("-"*70)

# Red + Volume elevated
red_vol = ((valid['day0_return'] < 0) & (valid['day0_vol_ratio'] > 1.5)).mean() * 100
print(f"\nRed Day + High Volume (>1.5x): {red_vol:.0f}%")

# Red + Closed High (accumulation)
red_close_high = ((valid['day0_return'] < 0) & (valid['day0_close_position'] > 0.5)).mean() * 100
print(f"Red Day + Closed Upper Half: {red_close_high:.0f}%")

# Multi-day red streak
if 'daym1_return' in valid.columns:
    two_red = ((valid['day0_return'] < 0) & (valid['daym1_return'] < 0)).mean() * 100
    print(f"2 consecutive red days (Day -1 & Day 0): {two_red:.0f}%")

print(f"\n" + "="*70)
print("🎯 DAY 0 PATTERN SUMMARY")
print("="*70)
print(f"""
What Day 0 typically looks like before a run:

1. Usually RED ({day0_red:.0f}% of the time)
2. Average return: {valid['day0_return'].mean():.1f}%
3. Closes at {close_pos:.0%} of daily range (middle-ish)
4. Volume typically {valid['day0_vol_ratio'].mean():.1f}x average
5. Only {vol_elevated:.0f}% have volume spike (>1.5x)

⚠️ KEY INSIGHT: Day 0 is NOT dramatic. It's a quiet red day.
   The volume spike comes on Day 1, not Day 0.
""")


🔬 TEST 2: WHAT DOES DAY 0 LOOK LIKE?

Analyzing 105 Day 0's before confirmed runs

----------------------------------------------------------------------
DAY 0 PRICE ACTION
----------------------------------------------------------------------

Day 0 is RED: 82%
Avg Day 0 return: -3.8%

Day 0 Close Position:
  Avg position in range: 33.2% (0=low, 1=high)
  Closed in bottom 30%: 52%
  Closed in TOP 30%: 12%

----------------------------------------------------------------------
DAY 0 VOLUME
----------------------------------------------------------------------

Day 0 volume > 1.5x average: 7%
Day 0 volume > 2.0x average: 2%
Avg Day 0 volume ratio: 0.9x

----------------------------------------------------------------------
COMBINED SIGNALS ON DAY 0
----------------------------------------------------------------------

Red Day + High Volume (>1.5x): 7%
Red Day + Closed Upper Half: 12%
2 consecutive red days (Day -1 & Day 0): 70%

🎯 DAY 0 PATTERN SUMMARY

What Day 0 typically looks like

---
# TEST 3: CAN WE IDENTIFY DAY 0 BY 3 PM?

**Question:** Is there a pattern we can detect BEFORE market close?

In [7]:
# CELL 7: TEST 3 - Day 0 Prediction Model

print("\n" + "="*70)
print("🔬 TEST 3: CAN WE IDENTIFY DAY 0 BY 3 PM?")
print("="*70)

# Build a scoring system
# These signals would be visible by 3 PM on Day 0

def calculate_day0_score(row):
    """Score a potential Day 0 based on available signals"""
    score = 0
    signals = []
    
    # Signal 1: Currently red for the day (+1)
    if row['day0_return'] < 0:
        score += 1
        signals.append('RED')
    
    # Signal 2: Volume elevated (+1)
    if row['day0_vol_ratio'] > 1.2:
        score += 1
        signals.append('VOL>1.2x')
    
    # Signal 3: Previous day also red (+1)
    if 'daym1_return' in row and pd.notna(row['daym1_return']) and row['daym1_return'] < 0:
        score += 1
        signals.append('PREV_RED')
    
    # Signal 4: Closing in upper half of range (recovery) (+1)
    if row['day0_close_position'] > 0.5:
        score += 1
        signals.append('CLOSE_HIGH')
    
    # Signal 5: Down more than 2% (oversold bounce potential) (+1)
    if row['day0_return'] < -2:
        score += 1
        signals.append('OVERSOLD')
    
    return score, signals

# Apply scoring to actual Day 0's
valid = runs_df.dropna(subset=['day0_return', 'day0_vol_ratio', 'day0_close_position']).copy()

scores = []
for _, row in valid.iterrows():
    score, signals = calculate_day0_score(row)
    scores.append({
        'ticker': row['ticker'],
        'score': score,
        'signals': signals,
        'run_gain': row['run_gain'],
        'day1_gap': row['day1_gap']
    })

score_df = pd.DataFrame(scores)

print(f"\nScored {len(score_df)} actual Day 0's (days before confirmed runs)")
print(f"\n" + "-"*70)
print("SCORE DISTRIBUTION ON ACTUAL DAY 0's")
print("-"*70)

print(f"\n| Score | Count | % of Day 0's | Avg Next Day Gap | Avg Run Gain |")
print(f"|-------|-------|--------------|------------------|--------------|")

for score in sorted(score_df['score'].unique()):
    subset = score_df[score_df['score'] == score]
    pct = len(subset) / len(score_df) * 100
    gap = subset['day1_gap'].mean()
    gain = subset['run_gain'].mean()
    print(f"| {score}     | {len(subset):5} | {pct:11.0f}% | {gap:+15.1f}% | {gain:+11.1f}% |")

print(f"\n" + "-"*70)
print("THE PROBLEM: FALSE POSITIVES")
print("-"*70)
print(f"""
This score tells us what actual Day 0's looked like.
But the REAL question is: How many NON-Day-0's also score high?

We need to test:
- How many random days score 3+?
- Of those, how many are actually followed by runs?

This is the FALSE POSITIVE rate - the trap.
""")


🔬 TEST 3: CAN WE IDENTIFY DAY 0 BY 3 PM?

Scored 105 actual Day 0's (days before confirmed runs)

----------------------------------------------------------------------
SCORE DISTRIBUTION ON ACTUAL DAY 0's
----------------------------------------------------------------------

| Score | Count | % of Day 0's | Avg Next Day Gap | Avg Run Gain |
|-------|-------|--------------|------------------|--------------|
| 0     |     4 |           4% |            +1.3% |       +48.7% |
| 1     |     8 |           8% |            +2.0% |       +37.8% |
| 2     |    27 |          26% |            +1.5% |       +35.9% |
| 3     |    49 |          47% |            +1.1% |       +43.8% |
| 4     |    16 |          15% |            +0.2% |       +30.3% |
| 5     |     1 |           1% |            +0.8% |       +19.9% |

----------------------------------------------------------------------
THE PROBLEM: FALSE POSITIVES
----------------------------------------------------------------------

This score t

In [8]:
# CELL 8: FALSE POSITIVE ANALYSIS

print("\n" + "="*70)
print("🔬 FALSE POSITIVE TEST: HOW MANY 'DAY 0' SIGNALS ARE TRAPS?")
print("="*70)

# For each ticker, score EVERY day and see how many "Day 0 signals" led to runs

all_day_scores = []

for ticker, df in all_data.items():
    if len(df) < 30:
        continue
    
    # Get this ticker's actual Day 1 dates
    ticker_runs = runs_df[runs_df['ticker'] == ticker]
    actual_day1_dates = set(ticker_runs['day1_date'].values) if len(ticker_runs) > 0 else set()
    
    # Score every day
    for i in range(25, len(df) - 1):  # Need history for avg vol, skip last day
        row = df.iloc[i]
        prev_row = df.iloc[i-1] if i > 0 else None
        next_row = df.iloc[i+1] if i < len(df) - 1 else None
        
        score = 0
        
        # Current day return
        day_return = row['Return'] if pd.notna(row['Return']) else 0
        vol_ratio = row['Vol_Ratio'] if pd.notna(row['Vol_Ratio']) else 1
        close_pos = row['Close_Position'] if pd.notna(row['Close_Position']) else 0.5
        
        # Signal 1: Red day
        if day_return < 0:
            score += 1
        
        # Signal 2: Volume elevated
        if vol_ratio > 1.2:
            score += 1
        
        # Signal 3: Previous day also red
        if prev_row is not None and pd.notna(prev_row['Return']) and prev_row['Return'] < 0:
            score += 1
        
        # Signal 4: Closing high
        if close_pos > 0.5:
            score += 1
        
        # Signal 5: Oversold
        if day_return < -2:
            score += 1
        
        # Did next day start a run?
        next_date = df.index[i+1] if i < len(df) - 1 else None
        led_to_run = next_date in actual_day1_dates if next_date is not None else False
        
        all_day_scores.append({
            'ticker': ticker,
            'date': df.index[i],
            'score': score,
            'led_to_run': led_to_run
        })

all_scores_df = pd.DataFrame(all_day_scores)

print(f"\nAnalyzed {len(all_scores_df)} total days across all tickers")

print(f"\n" + "-"*70)
print("FALSE POSITIVE RATE BY SCORE")
print("-"*70)
print(f"\n| Score | Days | Signaled 'Buy' | Led to Run | Hit Rate | False Pos |")
print(f"|-------|------|----------------|------------|----------|-----------|")

for score in sorted(all_scores_df['score'].unique()):
    subset = all_scores_df[all_scores_df['score'] == score]
    total = len(subset)
    hits = subset['led_to_run'].sum()
    hit_rate = hits / total * 100 if total > 0 else 0
    false_pos = 100 - hit_rate
    print(f"| {score}     | {total:4} | {'YES' if score >= 3 else 'NO':14} | {hits:10} | {hit_rate:7.1f}% | {false_pos:8.1f}% |")

print(f"\n" + "-"*70)
print("KEY FINDINGS")
print("-"*70)

# Score 3+ analysis
high_score = all_scores_df[all_scores_df['score'] >= 3]
high_score_hits = high_score['led_to_run'].sum()
high_score_total = len(high_score)
high_score_rate = high_score_hits / high_score_total * 100 if high_score_total > 0 else 0

print(f"\nIf we use Score 3+ as our 'buy signal':")
print(f"  Total 'signals': {high_score_total}")
print(f"  Actually led to run: {high_score_hits}")
print(f"  Hit Rate: {high_score_rate:.1f}%")
print(f"  FALSE POSITIVE RATE: {100-high_score_rate:.1f}%")

print(f"\n" + "="*70)
print("🎯 VERDICT: CAN WE IDENTIFY DAY 0?")
print("="*70)

if high_score_rate > 50:
    print(f"\n✅ YES - Score 3+ predicts runs {high_score_rate:.0f}% of the time")
elif high_score_rate > 30:
    print(f"\n⚠️ WEAK - Score 3+ only predicts runs {high_score_rate:.0f}% of the time")
    print(f"   False positive rate of {100-high_score_rate:.0f}% is too high.")
else:
    print(f"\n❌ NO - Score 3+ is basically random ({high_score_rate:.0f}% hit rate)")
    print(f"   Day 0 cannot be reliably identified with these signals.")


🔬 FALSE POSITIVE TEST: HOW MANY 'DAY 0' SIGNALS ARE TRAPS?

Analyzed 2549 total days across all tickers

----------------------------------------------------------------------
FALSE POSITIVE RATE BY SCORE
----------------------------------------------------------------------

| Score | Days | Signaled 'Buy' | Led to Run | Hit Rate | False Pos |
|-------|------|----------------|------------|----------|-----------|
| 0     |  105 | NO             |          0 |     0.0% |    100.0% |
| 1     |  632 | NO             |          0 |     0.0% |    100.0% |
| 2     | 1092 | NO             |          0 |     0.0% |    100.0% |
| 3     |  593 | YES            |          0 |     0.0% |    100.0% |
| 4     |  117 | YES            |          0 |     0.0% |    100.0% |
| 5     |   10 | YES            |          0 |     0.0% |    100.0% |

----------------------------------------------------------------------
KEY FINDINGS
----------------------------------------------------------------------

If we

---
# TEST 4: THE HONEST SUMMARY

What did we actually learn?

In [9]:
# CELL 9: Honest Summary

print("\n" + "="*70)
print("🐺 FENRIR MISSION SUMMARY: THE HONEST TRUTH")
print("="*70)

print(f"""
SAMPLE SIZE: {len(runs_df)} runs across {len(all_data)} tickers over 6 months
⚠️ This is LIMITED data. Take findings with healthy skepticism.

""")

print("-"*70)
print("WHAT WE FOUND:")
print("-"*70)

print(f"""
1. ENTRY TIMING:
   - Buying Day 0 close IS better than buying Day 1 open
   - But by how much? Need to verify with more data.
   - The gap captures some gains, but intraday fades are real.

2. DAY 0 CHARACTERISTICS:
   - Day 0 is usually RED ({day0_red:.0f}%)
   - Volume is NOT a reliable signal on Day 0
   - Day 0 looks like... any other red day

3. DAY 0 PREDICTION:
   - Simple signals have HIGH false positive rates
   - We cannot reliably identify Day 0 before it happens
   - The "tells" we thought existed are too weak

""")

print("-"*70)
print("WHAT WE STILL DON'T KNOW:")
print("-"*70)

print(f"""
1. WHY do runs start? (Catalyst? Rotation? Random?)
2. Are options flow/dark pool prints predictive?
3. Do sector leaders actually signal follower runs?
4. Is 6 months of data enough to draw conclusions?
5. Will these patterns hold going forward?

""")

print("-"*70)
print("THE UNCOMFORTABLE TRUTH:")
print("-"*70)

print(f"""
We found some patterns in historical data.
We have NOT proven they will work in the future.
We have NOT found a way to reliably predict Day 0.

The "dance" may exist, but we don't know the steps yet.
We need:
- Paper trading to test in real-time
- More data sources (options, insider trades)
- Longer time horizon
- Discipline to track and learn

This is Day 1 of the investigation, not the conclusion.
""")

print("\n" + "="*70)
print("🐺 NEXT STEPS")
print("="*70)

print(f"""
1. Start paper trading the current hypothesis
2. Log every signal, every trade, every outcome
3. Build data on what actually works vs what looked good in backtests
4. Add more data sources (options flow, Form 4's)
5. Be patient. The game is rigged. Learning the rules takes time.

THE WOLF DOESN'T CELEBRATE THE HUNT BEFORE THE KILL.
""")


🐺 FENRIR MISSION SUMMARY: THE HONEST TRUTH

SAMPLE SIZE: 125 runs across 25 tickers over 6 months
⚠️ This is LIMITED data. Take findings with healthy skepticism.


----------------------------------------------------------------------
WHAT WE FOUND:
----------------------------------------------------------------------

1. ENTRY TIMING:
   - Buying Day 0 close IS better than buying Day 1 open
   - But by how much? Need to verify with more data.
   - The gap captures some gains, but intraday fades are real.

2. DAY 0 CHARACTERISTICS:
   - Day 0 is usually RED (82%)
   - Volume is NOT a reliable signal on Day 0
   - Day 0 looks like... any other red day

3. DAY 0 PREDICTION:
   - Simple signals have HIGH false positive rates
   - We cannot reliably identify Day 0 before it happens
   - The "tells" we thought existed are too weak


----------------------------------------------------------------------
WHAT WE STILL DON'T KNOW:
-------------------------------------------------------------